In [1]:
import geopandas as gpd
import shapefile as shp
import matplotlib.pyplot as plt
import pandas as pd
import rtree
import seaborn as sns
import matplotlib
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

In [2]:
# utils
def get_3_plus(value):
    if value > 2: 
        return 1
    else:
        return 0

In [3]:
def get_projection(row, df):
    df['date'] = pd.to_datetime(df['date'])
    possible_dates = [row['offset_1'], row['offset_2'], row['offset_3']]
    try:
        actual = df.loc[(df['ADMIN2']==row['ADMIN2'])&(df['date']).isin(possible_dates)]
        return actual['FI_CS'].values[0]
    except:
        return None

In [4]:
cs = pd.read_csv('ethiopia_admin2_fewsnet_2020-05-05.csv')
ml1 = pd.read_csv('ethiopia_admin2_fewsnet_ML12020-05-05.csv')
ml2 = pd.read_csv('ethiopia_admin2_fewsnet_ML22020-05-05.csv')

In [5]:
cs

,Unnamed: 0,date,ADMIN0,ADMIN1,ADMIN2,CS
0,1373,2019-02-01,Ethiopia,SNNPR,Segen,2.0
1,6004,2019-02-01,Ethiopia,Oromia,Bale,2.0
2,1375,2019-02-01,Ethiopia,Oromia,Borena,2.0
3,3,2019-02-01,Ethiopia,Somali,Jarar,3.0
4,1930,2019-02-01,Ethiopia,Oromia,East Wellega,1.0
...,...,...,...,...,...,...
2883,4673,2018-12-01,Ethiopia,Amhara,North Gondar,99.0
2884,4648,2018-12-01,Ethiopia,Amhara,West Gojam,99.0
2885,4610,2018-12-01,Ethiopia,Amhara,Awi,99.0
2886,4663,2018-12-01,Ethiopia,Benshangul Gumuz,Pawe,99.0


In [6]:
ml1['date'] = pd.to_datetime(ml1['date'])
ml1['offset_1'] = ml1['date'] + pd.DateOffset(months=2)
ml1['offset_2'] = ml1['date'] + pd.DateOffset(months=3)
ml1['offset_3'] = ml1['date'] + pd.DateOffset(months=4)
ml1

,Unnamed: 0,date,ADMIN0,ADMIN1,ADMIN2,ML1,offset_1,offset_2,offset_3
0,0,2014-07-01,Ethiopia,SNNPR,Segen,1.0,2014-09-01,2014-10-01,2014-11-01
1,3944,2014-07-01,Ethiopia,Oromia,Borena,2.0,2014-09-01,2014-10-01,2014-11-01
2,2504,2014-07-01,Ethiopia,SNNPR,Sidama,1.0,2014-09-01,2014-10-01,2014-11-01
3,2515,2014-07-01,Ethiopia,SNNPR,Gamo Gofa,1.0,2014-09-01,2014-10-01,2014-11-01
4,2516,2014-07-01,Ethiopia,SNNPR,Gedeo,1.0,2014-09-01,2014-10-01,2014-11-01
...,...,...,...,...,...,...,...,...,...
2883,6229,2010-10-01,Ethiopia,SNNPR,Sheka,1.0,2010-12-01,2011-01-01,2011-02-01
2884,6232,2010-10-01,Ethiopia,Amhara,Awi,1.0,2010-12-01,2011-01-01,2011-02-01
2885,6248,2010-10-01,Ethiopia,Gambela,Mezhenger,1.0,2010-12-01,2011-01-01,2011-02-01
2886,6264,2010-10-01,Ethiopia,Benshangul Gumuz,Pawe,1.0,2010-12-01,2011-01-01,2011-02-01


In [7]:
ml2['date'] = pd.to_datetime(ml2['date'])
ml2['offset_1'] = ml2['date'] + pd.DateOffset(months=5)
ml2['offset_2'] = ml2['date'] + pd.DateOffset(months=6)
ml2['offset_3'] = ml2['date'] + pd.DateOffset(months=7)
ml2

,Unnamed: 0,date,ADMIN0,ADMIN1,ADMIN2,ML2,offset_1,offset_2,offset_3
0,0,2009-07-01,Ethiopia,SNNPR,Segen,3.0,2009-12-01,2010-01-01,2010-02-01
1,1,2009-07-01,Ethiopia,Oromia,Bale,3.0,2009-12-01,2010-01-01,2010-02-01
2,3850,2009-07-01,Ethiopia,Oromia,Borena,2.0,2009-12-01,2010-01-01,2010-02-01
3,3851,2009-07-01,Ethiopia,SNNPR,Sidama,2.0,2009-12-01,2010-01-01,2010-02-01
4,4,2009-07-01,Ethiopia,Somali,Afder,3.0,2009-12-01,2010-01-01,2010-02-01
...,...,...,...,...,...,...,...,...,...
2883,4414,2018-12-01,Ethiopia,Amhara,West Gojam,99.0,2019-05-01,2019-06-01,2019-07-01
2884,4417,2018-12-01,Ethiopia,Oromia,West Wellega,99.0,2019-05-01,2019-06-01,2019-07-01
2885,4420,2018-12-01,Ethiopia,SNNPR,Yem,99.0,2019-05-01,2019-06-01,2019-07-01
2886,4429,2018-12-01,Ethiopia,Benshangul Gumuz,Pawe,99.0,2019-05-01,2019-06-01,2019-07-01


In [8]:
cs['FI_CS'] = cs['CS'].apply(get_3_plus)
ml1['FI_ML1'] = ml1['ML1'].apply(get_3_plus)
ml2['FI_ML2'] = ml2['ML2'].apply(get_3_plus)

In [12]:
cs.head()

,Unnamed: 0,date,ADMIN0,ADMIN1,ADMIN2,CS,FI_CS
0,1373,2019-02-01,Ethiopia,SNNPR,Segen,2.0,0
1,6004,2019-02-01,Ethiopia,Oromia,Bale,2.0,0
2,1375,2019-02-01,Ethiopia,Oromia,Borena,2.0,0
3,3,2019-02-01,Ethiopia,Somali,Jarar,3.0,1
4,1930,2019-02-01,Ethiopia,Oromia,East Wellega,1.0,0


In [9]:
ml1['ACTUAL'] = ml1.apply(lambda x: get_projection(x, cs), axis=1)
#cs['ML2_PROJ'] = cs.apply(lambda x: get_previous_6(x, ml2), axis=1)

In [10]:
ml2['ACTUAL'] = ml2.apply(lambda x: get_projection(x, cs), axis=1)

In [11]:
ml1

,Unnamed: 0,date,ADMIN0,ADMIN1,ADMIN2,ML1,offset_1,offset_2,offset_3,FI_ML1,ACTUAL
0,0,2014-07-01,Ethiopia,SNNPR,Segen,1.0,2014-09-01,2014-10-01,2014-11-01,0,0.0
1,3944,2014-07-01,Ethiopia,Oromia,Borena,2.0,2014-09-01,2014-10-01,2014-11-01,0,0.0
2,2504,2014-07-01,Ethiopia,SNNPR,Sidama,1.0,2014-09-01,2014-10-01,2014-11-01,0,0.0
3,2515,2014-07-01,Ethiopia,SNNPR,Gamo Gofa,1.0,2014-09-01,2014-10-01,2014-11-01,0,0.0
4,2516,2014-07-01,Ethiopia,SNNPR,Gedeo,1.0,2014-09-01,2014-10-01,2014-11-01,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2883,6229,2010-10-01,Ethiopia,SNNPR,Sheka,1.0,2010-12-01,2011-01-01,2011-02-01,0,0.0
2884,6232,2010-10-01,Ethiopia,Amhara,Awi,1.0,2010-12-01,2011-01-01,2011-02-01,0,0.0
2885,6248,2010-10-01,Ethiopia,Gambela,Mezhenger,1.0,2010-12-01,2011-01-01,2011-02-01,0,0.0
2886,6264,2010-10-01,Ethiopia,Benshangul Gumuz,Pawe,1.0,2010-12-01,2011-01-01,2011-02-01,0,0.0


In [174]:
# classification_report (y_true, y_pred)
predictions = ml1.loc[ml1['ACTUAL']>-1]
print(classification_report(predictions['ACTUAL'], predictions['FI_ML1']))

              precision    recall  f1-score   support

         0.0       0.94      0.90      0.92      2211
         1.0       0.64      0.76      0.70       525

    accuracy                           0.87      2736
   macro avg       0.79      0.83      0.81      2736
weighted avg       0.88      0.87      0.88      2736



In [177]:
predictions = ml2.loc[ml2['ACTUAL']>-1]
print(classification_report(predictions['ACTUAL'], predictions['FI_ML2']))

              precision    recall  f1-score   support

         0.0       0.86      0.87      0.86      1567
         1.0       0.56      0.53      0.54       485

    accuracy                           0.79      2052
   macro avg       0.71      0.70      0.70      2052
weighted avg       0.79      0.79      0.79      2052

